In [ ]:
%pylab inline
%load_ext autoreload
from smartg import Smartg, Surface
from profile.profil import Profile, AeroOPAC
from luts import LUT, Idx, merge, read_lut_hdf, read_mlut_hdf
from smartg_view import semi_polar, smartg_view
from IPython.html.widgets.interaction import interact, interact_manual
from IPython.html import widgets
from IPython.display import clear_output

In [ ]:
# run SMART-G, and skip if file exists
job = Smartg('SMART-G-PP', THVDEG=30, wl=500., NBPHOTONS=1e9,
                    atm=Profile('afglt'),
                    surf=Surface(), skip_existing=True)
print job.output

In [ ]:
# read only one dataset
I_up = job.read('I_up (TOA)')

# read all datasets
R = job.read()
R.print_info()

In [ ]:
job.view()

In [ ]:
# basic display
semi_polar(I_up)

In [ ]:
# two plots side by side, including transects
fig = figure(figsize=(12, 6))
semi_polar(I_up, index=Idx(5.), rect=221, sub=223, swap=False, fig=fig)
semi_polar(I_up, index=Idx(30.), rect=222, sub=224, swap=True, fig=fig)  # reversed axes

In [ ]:
# interactive display
interact_manual(lambda vmax, phi:
                semi_polar(lut=I_up, index=Idx(phi), vmin=0, vmax=vmax),
                vmax=(0,1.,0.01),
                phi=(0, 180),
               )

-----
# Wavelength loop

In [ ]:
MLUTS = []
for wl in linspace(400, 800, 10):
    job = Smartg('SMART-G-PP', THVDEG=30,
                    wl=wl, NBPHOTONS=1e8,
                    atm=Profile('afglt'),
                    surf=Surface(), skip_existing=True
                   )
    print job.output

    MLUTS.append(job.read())

M = merge(MLUTS, ['LAMBDA'])
M['I_up (TOA)'].print_info()
semi_polar(M['I_up (TOA)'].sub[:,0,:], swap=True, index=12)

----
# Difference between two LUTs

In [ ]:
jobs = []
for ex in ['SMART-G-PP', 'SMART-G-SP']:
    job = Smartg(ex, THVDEG=30, wl=443., NBPHOTONS=1e9,
                         NBTHETA=20., NBPHI=20.,
                        atm=Profile('afglt'), skip_existing=True)
    jobs.append(job)
    
PP = jobs[0].read('I_up (TOA)')
SP = jobs[1].read('I_up (TOA)')

# show relative difference
semi_polar(100.*(SP-PP)/PP, Idx(45.), fig=figure(figsize(8, 8)), vmin=-2, vmax=2)

---
# Interactive simulation

In [ ]:
def simulate(surface=False, aerosols=False, wavelength=500., logN=8.):
    surf = {True:Surface(), False:None}[surface]
    aer = {True:AeroOPAC('maritime_polluted', 0.4, 550., layer_phase=-1),
           False:None}[aerosols]
    job = Smartg('SMART-G-PP', THVDEG=30,
             wl=wavelength, NBPHOTONS=10**logN,
             NBTHETA=20., NBPHI=20.,
             atm=Profile('afglt',aer=aer), surf=surf,
             overwrite=True)
    job.view()

interact_manual(simulate, wavelength=(400., 800.), logN=(5, 10, 0.01))
